<a href="https://colab.research.google.com/github/Arif-0123/Proyek_Analisis_Data/blob/main/Bike%20Sharing%20Dataset/submission/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyek Analisis Data: Bike Sharing Dataset
- Nama: Arif Hidayat
- Email: a.gaul0812@gmail.com
- Id Dicoding: kurokyshi

## Menentukan Pertanyaan Bisnis

- Bagaimana frekuensi prediksi jumlah penyewaan sepeda perjam atau perhari berdasarkan lingkungan dan musim?
- Bagaimana korelasi antara jumlah penyewaan sepeda dengan lingkungan dan musim?

For you information,
1. hour.csv: jumlah sepeda berbagi yang diagregasi secara per jam. Catatan: 17379 jam = 724 hari
2. day.csv: jumlah sepeda berbagi yang diagregasi secara harian. Catatan: 731 hari

jadi ditemukan perbedaan data yang tercatat perjam dan harian yaitu ada 7 hari yang tidak tercatat pada data perjam

## Menyiapkan semua library yang dibutuhkan

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn

## Data Wrangling

### Gathering Data

#### menampilkan semua dataset

In [3]:
day_df = pd.read_csv("https://raw.githubusercontent.com/Arif-0123/Proyek_Analisis_Data/main/Bike%20Sharing%20Dataset/submission/data/day.csv")
day_df.head()

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985
1,2,2011-01-02,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801
2,3,2011-01-03,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349
3,4,2011-01-04,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,108,1454,1562
4,5,2011-01-05,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,82,1518,1600


In [4]:
hour_df = pd.read_csv("https://raw.githubusercontent.com/Arif-0123/Proyek_Analisis_Data/main/Bike%20Sharing%20Dataset/submission/data/hour.csv")
hour_df.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


### Assessing Data

#### Menilai tabel `day_df`


In [5]:
day_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731 entries, 0 to 730
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   instant     731 non-null    int64  
 1   dteday      731 non-null    object 
 2   season      731 non-null    int64  
 3   yr          731 non-null    int64  
 4   mnth        731 non-null    int64  
 5   holiday     731 non-null    int64  
 6   weekday     731 non-null    int64  
 7   workingday  731 non-null    int64  
 8   weathersit  731 non-null    int64  
 9   temp        731 non-null    float64
 10  atemp       731 non-null    float64
 11  hum         731 non-null    float64
 12  windspeed   731 non-null    float64
 13  casual      731 non-null    int64  
 14  registered  731 non-null    int64  
 15  cnt         731 non-null    int64  
dtypes: float64(4), int64(11), object(1)
memory usage: 91.5+ KB


In [6]:
"""
Berikut Ini penjelasan variabel pada tabel day_df

instant: indeks catatan
dteday: tanggal
musim: musim (1: musim semi, 2: musim panas, 3: musim gugur, 4: musim dingin)
yr: tahun (0: 2011, 1: 2012)
mnth: bulan (1 hingga 12)
libur: hari cuaca adalah hari libur atau tidak (diambil dari http://dchr.dc.gov/page/holiday-schedule)
hari_kerja: jika hari bukan akhir pekan atau libur, nilainya adalah 1, jika tidak, nilainya adalah 0.
kondisi_cuaca:
1: Cerah, Sedikit awan, Sebagian berawan, Sebagian berawan
2: Berkabut + Berawan, Berkabut + Awan pecah, Berkabut + Sedikit awan, Berkabut
3: Salju Ringan, Hujan Ringan + Badai petir + Awan terpisah, Hujan Ringan + Awan terpisah
4: Hujan Lebat + Butiran Es + Badai petir + Berkabut, Salju + Kabut
suhu: Suhu terstandarisasi dalam Celsius. Nilai-nilai dibagi menjadi 41 (maksimal)
atemp: Suhu perasaan terstandarisasi dalam Celsius. Nilai-nilai dibagi menjadi 50 (maksimal)
kelembaban: Kelembaban terstandarisasi. Nilai-nilai dibagi menjadi 100 (maksimal)
kecepatan_angin: Kecepatan angin terstandarisasi. Nilai-nilai dibagi menjadi 67 (maksimal)
kasual: jumlah pengguna kasual
terdaftar: jumlah pengguna terdaftar
cnt: jumlah total sepeda yang disewakan termasuk pengguna kasual dan terdaftar

"""

'\nBerikut Ini penjelasan variabel pada tabel day_df\n\ninstant: indeks catatan \ndteday: tanggal \nmusim: musim (1: musim semi, 2: musim panas, 3: musim gugur, 4: musim dingin)\nyr: tahun (0: 2011, 1: 2012)\nmnth: bulan (1 hingga 12)\nlibur: hari cuaca adalah hari libur atau tidak (diambil dari http://dchr.dc.gov/page/holiday-schedule)\nhari_kerja: jika hari bukan akhir pekan atau libur, nilainya adalah 1, jika tidak, nilainya adalah 0.\nkondisi_cuaca:\n1: Cerah, Sedikit awan, Sebagian berawan, Sebagian berawan\n2: Berkabut + Berawan, Berkabut + Awan pecah, Berkabut + Sedikit awan, Berkabut\n3: Salju Ringan, Hujan Ringan + Badai petir + Awan terpisah, Hujan Ringan + Awan terpisah\n4: Hujan Lebat + Butiran Es + Badai petir + Berkabut, Salju + Kabut\nsuhu: Suhu terstandarisasi dalam Celsius. Nilai-nilai dibagi menjadi 41 (maksimal)\natemp: Suhu perasaan terstandarisasi dalam Celsius. Nilai-nilai dibagi menjadi 50 (maksimal)\nkelembaban: Kelembaban terstandarisasi. Nilai-nilai dibagi men

In [7]:
day_df.isna().sum()

instant       0
dteday        0
season        0
yr            0
mnth          0
holiday       0
weekday       0
workingday    0
weathersit    0
temp          0
atemp         0
hum           0
windspeed     0
casual        0
registered    0
cnt           0
dtype: int64

In [8]:
print("duplikasi data : ", day_df.duplicated().sum())

duplikasi data :  0


In [9]:
day_df.describe()

,instant,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
count,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000
mean,366.000000,2.496580,0.500684,6.519836,0.028728,2.997264,0.683995,1.395349,0.495385,0.474354,0.627894,0.190486,848.176471,3656.172367,4504.348837
std,211.165812,1.110807,0.500342,3.451913,0.167155,2.004787,0.465233,0.544894,0.183051,0.162961,0.142429,0.077498,686.622488,1560.256377,1937.211452
min,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.059130,0.079070,0.000000,0.022392,2.000000,20.000000,22.000000
25%,183.500000,2.000000,0.000000,4.000000,0.000000,1.000000,0.000000,1.000000,0.337083,0.337842,0.520000,0.134950,315.500000,2497.000000,3152.000000
50%,366.000000,3.000000,1.000000,7.000000,0.000000,3.000000,1.000000,1.000000,0.498333,0.486733,0.626667,0.180975,713.000000,3662.000000,4548.000000
75%,548.500000,3.000000,1.000000,10.000000,0.000000,5.000000,1.000000,2.000000,0.655417,0.608602,0.730209,0.233214,1096.000000,4776.500000,5956.000000
max,731.000000,4.000000,1.000000,12.000000,1.000000,6.000000,1.000000,3.000000,0.861667,0.840896,0.972500,0.507463,3410.000000,6946.000000,8714.000000


In [10]:
day_df.nunique()

instant       731
dteday        731
season          4
yr              2
mnth           12
holiday         2
weekday         7
workingday      2
weathersit      3
temp          499
atemp         690
hum           595
windspeed     650
casual        606
registered    679
cnt           696
dtype: int64

#### Menilai tabel `hour_df`

In [11]:
hour_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17379 entries, 0 to 17378
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   instant     17379 non-null  int64  
 1   dteday      17379 non-null  object 
 2   season      17379 non-null  int64  
 3   yr          17379 non-null  int64  
 4   mnth        17379 non-null  int64  
 5   hr          17379 non-null  int64  
 6   holiday     17379 non-null  int64  
 7   weekday     17379 non-null  int64  
 8   workingday  17379 non-null  int64  
 9   weathersit  17379 non-null  int64  
 10  temp        17379 non-null  float64
 11  atemp       17379 non-null  float64
 12  hum         17379 non-null  float64
 13  windspeed   17379 non-null  float64
 14  casual      17379 non-null  int64  
 15  registered  17379 non-null  int64  
 16  cnt         17379 non-null  int64  
dtypes: float64(4), int64(12), object(1)
memory usage: 2.3+ MB


In [12]:
"""
Berikut Ini penjelasan variabel pada tabel hour_df

instant: indeks catatan
dteday: tanggal
musim: musim (1: musim semi, 2: musim panas, 3: musim gugur, 4: musim dingin)
yr: tahun (0: 2011, 1: 2012)
mnth: bulan (1 hingga 12)
hr: jam (0 hingga 23)
libur: hari cuaca adalah hari libur atau tidak (diambil dari http://dchr.dc.gov/page/holiday-schedule)
hari_kerja: jika hari bukan akhir pekan atau libur, nilainya adalah 1, jika tidak, nilainya adalah 0.
kondisi_cuaca:
1: Cerah, Sedikit awan, Sebagian berawan, Sebagian berawan
2: Berkabut + Berawan, Berkabut + Awan pecah, Berkabut + Sedikit awan, Berkabut
3: Salju Ringan, Hujan Ringan + Badai petir + Awan terpisah, Hujan Ringan + Awan terpisah
4: Hujan Lebat + Butiran Es + Badai petir + Berkabut, Salju + Kabut
suhu: Suhu terstandarisasi dalam Celsius. Nilai-nilai dibagi menjadi 41 (maksimal)
atemp: Suhu perasaan terstandarisasi dalam Celsius. Nilai-nilai dibagi menjadi 50 (maksimal)
kelembaban: Kelembaban terstandarisasi. Nilai-nilai dibagi menjadi 100 (maksimal)
kecepatan_angin: Kecepatan angin terstandarisasi. Nilai-nilai dibagi menjadi 67 (maksimal)
kasual: jumlah pengguna kasual
terdaftar: jumlah pengguna terdaftar
cnt: jumlah total sepeda yang disewakan termasuk pengguna kasual dan terdaftar

"""

'\nBerikut Ini penjelasan variabel pada tabel hour_df\n\ninstant: indeks catatan\ndteday: tanggal\nmusim: musim (1: musim semi, 2: musim panas, 3: musim gugur, 4: musim dingin)\nyr: tahun (0: 2011, 1: 2012)\nmnth: bulan (1 hingga 12)\nhr: jam (0 hingga 23)\nlibur: hari cuaca adalah hari libur atau tidak (diambil dari http://dchr.dc.gov/page/holiday-schedule)\nhari_kerja: jika hari bukan akhir pekan atau libur, nilainya adalah 1, jika tidak, nilainya adalah 0.\nkondisi_cuaca:\n1: Cerah, Sedikit awan, Sebagian berawan, Sebagian berawan\n2: Berkabut + Berawan, Berkabut + Awan pecah, Berkabut + Sedikit awan, Berkabut\n3: Salju Ringan, Hujan Ringan + Badai petir + Awan terpisah, Hujan Ringan + Awan terpisah\n4: Hujan Lebat + Butiran Es + Badai petir + Berkabut, Salju + Kabut\nsuhu: Suhu terstandarisasi dalam Celsius. Nilai-nilai dibagi menjadi 41 (maksimal)\natemp: Suhu perasaan terstandarisasi dalam Celsius. Nilai-nilai dibagi menjadi 50 (maksimal)\nkelembaban: Kelembaban terstandarisasi. 

In [13]:
hour_df.isna().sum()

instant       0
dteday        0
season        0
yr            0
mnth          0
hr            0
holiday       0
weekday       0
workingday    0
weathersit    0
temp          0
atemp         0
hum           0
windspeed     0
casual        0
registered    0
cnt           0
dtype: int64

In [14]:
print("duplikasi data : ", hour_df.duplicated().sum())

duplikasi data :  0


In [15]:
hour_df.describe()

,instant,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
count,17379.0000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000
mean,8690.0000,2.501640,0.502561,6.537775,11.546752,0.028770,3.003683,0.682721,1.425283,0.496987,0.475775,0.627229,0.190098,35.676218,153.786869,189.463088
std,5017.0295,1.106918,0.500008,3.438776,6.914405,0.167165,2.005771,0.465431,0.639357,0.192556,0.171850,0.192930,0.122340,49.305030,151.357286,181.387599
min,1.0000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,4345.5000,2.000000,0.000000,4.000000,6.000000,0.000000,1.000000,0.000000,1.000000,0.340000,0.333300,0.480000,0.104500,4.000000,34.000000,40.000000
50%,8690.0000,3.000000,1.000000,7.000000,12.000000,0.000000,3.000000,1.000000,1.000000,0.500000,0.484800,0.630000,0.194000,17.000000,115.000000,142.000000
75%,13034.5000,3.000000,1.000000,10.000000,18.000000,0.000000,5.000000,1.000000,2.000000,0.660000,0.621200,0.780000,0.253700,48.000000,220.000000,281.000000
max,17379.0000,4.000000,1.000000,12.000000,23.000000,1.000000,6.000000,1.000000,4.000000,1.000000,1.000000,1.000000,0.850700,367.000000,886.000000,977.000000


In [16]:
hour_df.nunique()

instant       17379
dteday          731
season            4
yr                2
mnth             12
hr               24
holiday           2
weekday           7
workingday        2
weathersit        4
temp             50
atemp            65
hum              89
windspeed        30
casual          322
registered      776
cnt             869
dtype: int64

### Cleaning Data

ditemukan beberapa anomali dalam data day_df dan hour_df
yaitu :

1. dteday type datanya salah yang seharusnya datetime pada day_df dan hour_df
2. weathersit day_df seharusnya nilai maksimalnya 4 karena berkorelasi dengan hour_df yang mengakibatkan anomali data


## Exploratory Data Analysis (EDA)

### Explore ...

## Visualization & Explanatory Analysis

### Pertanyaan 1:

### Pertanyaan 2:

## Conclusion

- Conclution pertanyaan 1
- conclution pertanyaan 2